In [13]:
from pymongo import MongoClient

In [14]:
mongo = MongoClient()

# database
soccer_db = mongo['soccer-tweets']

# collection
tweets = soccer_db['users']

In [15]:
# total doc size
tweets.count_documents(filter={})

11188

In [16]:
# sample row
tweets.find_one(filter={})

{'_id': ObjectId('578ffa8e7eb9513f4f55a935'),
 'user_name': 'koteras',
 'retweet_count': 0,
 'tweet_followers_count': 461,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'coordinates': None,
 'tweet_mentioned_count': 1,
 'tweet_ID': '755891629932675072',
 'tweet_text': 'RT @ochocinco: I beat them all for 10 straight hours #FIFA16KING  https://t.co/BFnV6jfkBL',
 'user': {'CreatedAt': datetime.datetime(2011, 12, 27, 9, 4, 1),
  'FavouritesCount': 5223,
  'FollowersCount': 461,
  'FriendsCount': 619,
  'UserId': 447818090,
  'Location': '501'}}

#### Q1: How many tweets have location not null?

In [17]:
# coordinates: that exist and has Non Null value
tweets.count_documents({"coordinates": {'$ne' : None}})

0

In [18]:
# user.Location: that exist and has Non Null value
tweets.count_documents({"user.Location": {'$ne' : None}})

6937

####  Q2: How many people have more followers than friends?

In [19]:
len(tweets.distinct('user.UserId', {'$expr' : {'$gt' : ['$user.FollowersCount', '$user.FriendsCount']}}))

4971

OR

In [20]:
pipeline = [
    
    {'$match': {'$expr' : {'$gt' : ['$user.FollowersCount', '$user.FriendsCount']}}},
    
    {'$group' : { '_id' : "$user.UserId"}},
    
    {'$group': {'_id' : None,  'count': { '$sum': 1 }}}
]

results_ = tweets.aggregate(pipeline)

results = list(results_)

print(results[0]['count'])

4971


#### Q3: Return text of tweets which have the string "http://" ?

In [21]:
results_ = tweets.find({'tweet_text': {'$regex' : '.*http://*.'}}, {'tweet_text':1})

In [22]:
for row in results_:
    print(row)
    break

{'_id': ObjectId('57965376c381590a1c83bbc4'), 'tweet_text': 'RT @BWGotheem: Top 15 Sexiest Woman of the World Cup😍👌🏼😱\n\nhttp://t.co/jRQy5AkVtW http://t.co/aMuXZ3gcWy'}


#### Q4: Return all the tweets which contain text "England" but not "UEFA" ?

In [23]:
filter_ = {
    
    '$and': [
        {'tweet_text': {'$regex' : '.*England*.', '$options': 'i'}},
        {'tweet_text': {'$not' : {'$regex' : '.*UEFA*.'}}}
    ]
}

results_ = tweets.find(filter_)

for row in results_:
    print(row)
    break

{'_id': ObjectId('57965ef7c38159118f94f8b4'), 'user_name': 'fifafoot', 'retweet_count': 0, 'tweet_followers_count': 838, 'source': '<a href="http://twitterfeed.com" rel="nofollow">twitterfeed</a>', 'coordinates': None, 'tweet_mentioned_count': 0, 'tweet_ID': '757648709446270976', 'tweet_text': '#fifa #foot Newly appointed England manager Sam Allardyce: BURTON-UPON-TRENT, ENGLAND... https://t.co/FpqWudQM1Y https://t.co/2L0uFYRTqr', 'user': {'CreatedAt': datetime.datetime(2009, 9, 9, 19, 56, 14), 'FavouritesCount': 0, 'FollowersCount': 838, 'FriendsCount': 285, 'UserId': 72935312, 'Location': 'London'}}


#### Q5: Get all the tweets from the location "Ireland" and contains the string "UEFA"?

In [24]:
filter_ = {
    
    '$and': [
        {'user.Location': {'$regex' : '.*Ireland*.', '$options' : 'i'}},
        {'tweet_text': {'$regex' : '.*UEFA*.'}}
    ]
}


results_ = tweets.find(filter_)

for row in results_:
    print(row)
    break

{'_id': ObjectId('578ffb9a7eb951401527b554'), 'user_name': 'futbol__97', 'retweet_count': 0, 'tweet_followers_count': 400, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'coordinates': None, 'tweet_mentioned_count': 1, 'tweet_ID': '755892754287304705', 'tweet_text': 'RT @DundalkFC: Full Time FH 2-2 Dundalk, Dundalk qualify to the next round of the UEFA Champions League!', 'user': {'CreatedAt': datetime.datetime(2014, 7, 14, 0, 48, 21), 'FavouritesCount': 19708, 'FollowersCount': 400, 'FriendsCount': 398, 'UserId': -1586166401, 'Location': 'Ireland '}}
